<a href="https://colab.research.google.com/github/Karasiari/Graphs/blob/main/Clearing.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
%pip install "git+https://github.com/Karasiari/HuGraphFinal.git"

  Cloning https://github.com/Karasiari/HuGraphFinal.git to /tmp/pip-req-build-kith2aad
  Running command git clone --filter=blob:none --quiet https://github.com/Karasiari/HuGraphFinal.git /tmp/pip-req-build-kith2aad
  Resolved https://github.com/Karasiari/HuGraphFinal.git to commit 6d87c30b9c34a5a9ec1c2e674bc2da5e96b007bd
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Created wheel for hugraphfin: filename=hugraphfin-0.1.0-py3-none-any.whl size=29091 sha256=1d5d5789a9d6a9d7618575ba0ee564694513af49fddcfa67276b7b7754cf75c8
  Stored in directory: /tmp/pip-ephem-wheel-cache-ir8yah6b/wheels/46/83/89/572a6db8e6320eb877bf6ced236fa61f993538a12c5e43c72e
Successfully built hugraphfin


In [2]:
import pandas as pd
import numpy as np
import networkx as nx
import os
from google.colab import files

# две функции - читалки hu графов в мультиграфы

BITRATE_DENOMINATOR = 100

def hu_csv_to_graphs(path, demands_path, capacity_path):
  Network = pd.read_csv(path, header=None, names = ['id', 'source', 'target', 'length'])
  Traffic = pd.read_csv(demands_path, header=None, names = ['id', 'source', 'target', 'bitrate'])
  Capacity = pd.read_csv(capacity_path, header=None, names = ['name', 'value'])
  network = Network.iloc[1:].copy()
  traffic = Traffic.iloc[1:].copy()
  capacity = Capacity.iloc[1:].copy()

  network['source'] = network['source'].astype(int)
  network['target'] = network['target'].astype(int)
  traffic['source'] = traffic['source'].astype(int)
  traffic['target'] = traffic['target'].astype(int)
  traffic['bitrate'] = traffic['bitrate'].astype(float) / BITRATE_DENOMINATOR
  capacity_value = float(capacity[capacity['name']=='NumberOfWavelengths']['value'])

  sources = network['source'].tolist()
  targets = network['target'].tolist()
  sources_traffic = traffic['source'].tolist()
  targets_traffic = traffic['target'].tolist()
  bitrates_traffic = traffic['bitrate'].tolist()

  unique_vertices = set()
  for source, target in zip(sources, targets):
    unique_vertices.add(source)
    unique_vertices.add(target)
  vertex_mapping = {old: new for new, old in enumerate(unique_vertices)}
  num_vertices = len(unique_vertices)

  adj_edges = []
  traffic_edges = []

  for source, target in zip(sources, targets):
    new_source = vertex_mapping[source]
    new_target = vertex_mapping[target]
    adj_edges.append((new_source, new_target, {"capacity": capacity_value}))
  for source, target, bitrate in zip(sources_traffic, targets_traffic, bitrates_traffic):
    new_source = vertex_mapping[source]
    new_target = vertex_mapping[target]
    traffic_edges.append((new_source, new_target, {"weight": bitrate}))

  adj_graph = nx.MultiGraph()
  adj_graph.add_edges_from(adj_edges)
  traffic_graph = nx.MultiDiGraph()
  traffic_graph.add_nodes_from(range(adj_graph.number_of_nodes()))
  traffic_graph.add_edges_from(traffic_edges)

  return adj_graph, traffic_graph, capacity_value

def get_hu_graphs(base_path, specified_graphs, specified):
  Graphs = {}
  csv_tables = []
  for folder_name in os.listdir(base_path):
    if folder_name in specified_graphs or not specified:
      folder_path = os.path.join(base_path, folder_name)
      csv_path = os.path.join(folder_path, 'links.csv')
      csv_path_demands = os.path.join(folder_path, 'demands.csv')
      csv_path_capacity = os.path.join(folder_path, 'params.csv')
      csv_tables.append(folder_name)

      adj_graph, traffic_graph, capacity_value = hu_csv_to_graphs(csv_path, csv_path_demands, csv_path_capacity)
      Graphs[folder_name] = {'adj_graph': adj_graph, 'traffic_graph': traffic_graph, 'capacity_value': capacity_value}

  return Graphs

path_to_folder = '/content/drive/MyDrive/Кола'

In [14]:
hu_graphs_names = ['cola_t8']#, 'cola_t3']
hu_graphs = get_hu_graphs(path_to_folder, hu_graphs_names, True)

/tmp/ipython-input-3554038479.py:24: FutureWarning: Calling float on a single element Series is deprecated and will raise a TypeError in the future. Use float(ser.iloc[0]) instead
  capacity_value = float(capacity[capacity['name']=='NumberOfWavelengths']['value'])


In [15]:
from graph_exps import HuGraphForExps
from graph_exps.exp import *

for name, graph in hu_graphs.items():
  Graph = HuGraphForExps(graph['adj_graph'], graph['traffic_graph'])
  additional_resources = [80.0, 80.0, 80.0]
  allocation_types = ['alpha', 'random_alpha', 'random']
  results = expand_test_for_graph(Graph, additional_resources, allocation_types, tries_for_allocation=5)

Processing all edges:   0%|          | 0/63 [00:00<?, ?it/s]

Solving initial MCF:   0%|          | 0/3 [00:00<?, ?it/s]

Solving remaining network MCFPs for alpha:   0%|          | 0/63 [00:00<?, ?it/s]

Solving remaining network MCFPs for random_alpha:   0%|          | 0/63 [00:00<?, ?it/s]

Solving remaining network MCFPs for random:   0%|          | 0/63 [00:00<?, ?it/s]

Processing allocation:   0%|          | 0/15 [00:00<?, ?it/s]

In [16]:
results

allocation solved rerouted volume gamma for failed (0, 1)  \
alpha        1              True             173                     inf   
             2              True             173                     inf   
             3              True             173                     inf   
             4              True             173                     inf   
             5              True             173                     inf   
random_alpha 1              True             173                     inf   
             2              True             173                     inf   
             3              True             173                     inf   
             4              True             173                     inf   
             5              True             173                     inf   
random       1              True             173                     inf   
             2              True             173                     inf   
             3              True             173                     inf   
             4              True             173                     inf   
             5              True             173                     inf   

               gamma for failed (0, 3) gamma for failed (0, 19)  \
alpha        1                    31.0                     57.5   
             2                    31.0                     57.5   
             3                    31.0                     57.5   
             4                    31.0                     57.5   
             5                    31.0                     57.5   
random_alpha 1                    38.0                    37.75   
             2                    38.0                    37.75   
             3                    38.0                    37.75   
             4                    38.0                    37.75   
             5                    38.0                    37.75   
random       1                    38.0                    37.75   
             2                    38.0                    37.75   
             3                    38.0                    37.75   
             4                    38.0                    37.75   
             5                    38.0                    37.75   

               gamma for failed (0, 21) gamma for failed (2, 3)  \
alpha        1                38.833333                     inf   
             2                38.833333                     inf   
             3                38.833333                     inf   
             4                38.833333                     inf   
             5                38.833333                     inf   
random_alpha 1                     30.6                     inf   
             2                     30.6                     inf   
             3                     30.6                     inf   
             4                     30.6                     inf   
             5                     30.6                     inf   
random       1                     30.6                     inf   
             2                     30.6                     inf   
             3                     30.6                     inf   
             4                     30.6                     inf   
             5                     30.6                     inf   

               gamma for failed (2, 4) gamma for failed (3, 4)  \
alpha        1                     inf                    47.0   
             2                     inf                    47.0   
             3                     inf                    47.0   
             4                     inf                    47.0   
             5                     inf                    47.0   
random_alpha 1                     inf                    78.0   
             2                     inf                    78.0   
             3                     inf                    78.0   
             4                     inf                    